```python
df.groupby('QLabel').filter(lambda x: x['year'].nunique() <= 3)
```

In [1]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

d:\Software\Anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [115]:
df=pd.read_csv("r1.csv")

CPCleanedUnique=df.CPCleaned.unique()

def collapse(cp):
    for i in df[df.CPCleaned==cp].QCleaned.unique():
        print(i,"\n",tabCountry(i),"\n---------------------------\n", getQNum(i),sep="")
        
def getOriginalQ(cleanedQ):
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

with open("grpdQ.txt","r") as txt:
    grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

In [116]:
# should be:
# 0    820466
# 1    233138
df.cleaned.value_counts()

0    820466
1    233138
Name: cleaned, dtype: int64

In [120]:
with open("grpdRes.txt", "r") as gR:
    grl=gR.read().split("\n")[:-2]

# tested, works:
for x in grl:
    idx= int(x.split(":")[0].split(" ")[-1])
    replace(grpdQ[idx])

---

In [122]:
#0:
df[df.CPCleaned==CPCleanedUnique[0]].QCleaned.value_counts()

Accessable unlocked emergency exits                                                  6476
Periodic emergency drills                                                            6476
Fire detection and alarm system                                                      6476
Does the workplace have adequate fire-fighting equipment?                            6476
At least 2 possible emergency exits                                                  6476
Emergency exits/escape routes clearly marked                                         6269
Flammable materials safely stored                                                    6225
Are possible sources of ignition appropriately safeguarded?                          6212
Trained workers on fire fighting                                                     5637
Escape routes locked                                                                 4994
Has the employer elaborated and implemented an emergency plan?                       1921
Does the e

In [123]:
replace(["Trained workers on fire fighting","Has the employer appointed and trained a fire management team?"])

In [124]:
#0after:
df[df.CPCleaned==CPCleanedUnique[0]].QCleaned.value_counts()

Accessable unlocked emergency exits                                                  6476
Fire detection and alarm system                                                      6476
At least 2 possible emergency exits                                                  6476
Periodic emergency drills                                                            6476
Does the workplace have adequate fire-fighting equipment?                            6476
Trained workers on fire fighting                                                     6470
Emergency exits/escape routes clearly marked                                         6269
Flammable materials safely stored                                                    6225
Are possible sources of ignition appropriately safeguarded?                          6212
Escape routes locked                                                                 4994
Has the employer elaborated and implemented an emergency plan?                       1921
Does the e

drop: <4994

-----


In [135]:
#1:
# undo("Does the employer pay workers correctly for all overtime hours worked at night?")
df[df.CPCleaned==CPCleanedUnique[1]].QCleaned.value_counts()

Payment of overtime                                                                                                          6476
Does the employer pay workers correctly for paid public holidays?                                                            4305
Does the employer pay all workers the correct meal allowance or give them a reasonable free meal when they work overtime?    2503
Does the employer pay workers double their normal wage for all hours worked on weekly rest days?                             2503
Does the employer pay workers double their normal wage for all overtime hours worked at night?                               2238
Does the employer pay workers correctly for all overtime hours worked at night?                                              1919
Does the employer pay workers correctly for all overtime hours worked on weekly rest days?                                   1802
Does the employer provide meals and drinks of at least 1,400 calories to workers working o

**combine**:
other pay for overtime=
[
Does the employer pay workers double their normal wage for all hours worked on weekly rest days?                      
Does the employer pay workers correctly for all overtime hours worked on weekly rest days?                            
Does the employer provide meals and drinks of at least 1,400 calories to workers working overtime for 3 hours or more?
Does the employer pay workers 150% of their normal wage for overtime worked on weekly rest days and public holidays?  
Does the employer pay workers 100% more than their normal wage for all overtime hours worked on weekly rest days?   ]  

In [ ]:
#*
replace(["Does the employer pay workers correctly for paid public holidays?",\
         "Does the employer pay workers 150% of their normal wage for overtime worked on weekly rest days and public holidays?"])

In [286]:
getOriginalQ("Does the employer pay workers correctly for paid public holidays?")

Does the employer pay workers correctly for paid public holidays?                                                                                             3550
Does the employer pay all workers correctly for work on public holidays?                                                                                      2503
Does the employer pay workers for legally mandated paid public holidays?                                                                                      1802
Does the employer pay workers correctly for all overtime hours worked on public holidays?                                                                     1802
Does the employer pay workers correctly for festival holidays (including compensatory festival holidays when workers work on festival holidays)?               584
Does the employer provide workers at least 11 festival holidays per year (or compensatory and substitute holidays when workers work on festival holidays)?     566
Does the employer pay 

In [66]:
for i in df[df.CPCleaned==CPCleanedUnique[1]].QCleaned.unique():
    print(i,"\n",df[df.QCleaned==i].Country.unique())

Payment of overtime 
 ['Vietnam' 'Indonesia' 'Jordan' 'Nicaragua' 'Haiti' 'Cambodia'
 'Bangladesh']
Does the employer pay workers correctly for all overtime hours worked at night? 
 ['Vietnam' 'Nicaragua']
Does the employer pay workers correctly for all overtime hours worked on public holidays? 
 ['Vietnam']
Does the employer pay workers correctly for all overtime hours worked on weekly rest days? 
 ['Vietnam']
Does the employer provide meals and drinks of at least 1,400 calories to workers working overtime for 3 hours or more? 
 ['Indonesia']
Does the employer pay workers 150% of their normal wage for overtime worked on weekly rest days and public holidays? 
 ['Jordan']
Does the employer pay workers 100% more than their normal wage for all overtime hours worked on weekly rest days? 
 ['Nicaragua']
Does the employer pay workers double their normal wage for all overtime hours worked at night? 
 ['Cambodia']
Does the employer pay all workers the correct meal allowance or give them a reas

In [217]:
len(df[df.QCleaned=='Does the employer pay workers the religious holiday allowance?'])

833

In [215]:
df[df.QCleaned.str.contains("holiday")].QCleaned.unique()

array(['Does the employer pay workers correctly for paid public holidays?',
       'Does the employer pay workers the religious holiday allowance?',
       'Does the employer pay workers 150% of their normal wage for overtime worked on weekly rest days and public holidays?'],
      dtype=object)

In [316]:
#2:
# undo("Does the employer terminate workers who are on maternity leave or force them to resign?")
# undo("Is maternity leave excluded from workers' period of continuous service?")
# undo("Terminate workers due to pregnancy or maternity leave")
# replace(["Does the employer dismiss workers or force them to resign if they are pregnant, on maternity leave or nursing? [differentiation question]",
# "Does the employer dismiss workers or force them to resign if they are pregnant, on maternity leave or nursing?"])

# replace(["Does the employer terminate workers who are pregnant or on maternity leave or force them to resign?",
#          "Does the employer terminate workers who are on maternity leave or force them to resign?"])
df[df.CPCleaned==CPCleanedUnique[2]].QCleaned.value_counts()

Gender discrimination on conditions of work                                                                                                                                   6476
Sexual harassment                                                                                                                                                             6476
Is gender a factor in decisions regarding pay?                                                                                                                                6315
Does the employer require pregnancy tests or use of contraceptives as a condition of employment?                                                                              6209
Is an applicant's gender a factor in hiring decisions?                                                                                                                        6002
Is gender a factor in decisions regarding opportunities for promotion or access to training?             

In [330]:
#* drop:
df.drop(df[(df.Q=="Does the employer terminate workers who are pregnant or on maternity leave or force them to resign?") & (df.Country=="Cambodia")])

,Country,FactoryAssessedID,AssesmentDate,QuestID,CP,Q,Finding,QCleaned,CPCleaned,cleaned
23042,Cambodia,1151,04/05/2015,8984,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
23260,Cambodia,747,05/05/2015,9017,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
23882,Cambodia,1165,19/05/2015,9061,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
24604,Cambodia,755,05/05/2015,9085,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
25856,Cambodia,1541,07/05/2015,9150,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
...,...,...,...,...,...,...,...,...,...,...
59143,Cambodia,1799,16/06/2015,10452,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
60430,Cambodia,1172,06/07/2015,10500,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
61057,Cambodia,1450,22/06/2015,10515,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1
69183,Cambodia,871,13/07/2015,10833,Gender,Does the employer terminate workers who are pr...,0,Does the employer terminate workers who are on...,Gender,1


In [320]:
collapse(CPCleanedUnique[2])

Gender discrimination on conditions of work
Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
---------------------------
6476
Does the employer terminate workers due to the worker's pregnancy, maternity leave, marriage, or breast-feeding of a child under 12 months of age or force them to resign?
Vietnam    721
Name: Country, dtype: int64
---------------------------
721
Sexual harassment
Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
---------------------------
6476
Is gender a factor in decisions regarding pay?
Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Nicaragua      119
Name: Country, dtype: int64
---------------------------
6315
Is gender a factor in decisions regarding opportunities for promotion or access to

In [319]:
4124+1054+721+480+161

6540

In [312]:
df[(df.Country=="Jordan") & (df.Q.str.contains("gender"))].QCleaned.unique()

array(['Gender discrimination on conditions of work',
       "Do job announcements refer to the applicant's gender?",
       'Is gender a factor in decisions regarding pay?'], dtype=object)

In [310]:
tabCountry("Is an applicant's gender a factor in hiring decisions?")

Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Haiti          161
Nicaragua      119
Name: Country, dtype: int64

In [153]:
for i in df[df.CPCleaned==CPCleanedUnique[2]].QCleaned.unique():
    print(i,"\n",tabCountry(i))

Gender discrimination on conditions of work 
 Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Does the employer terminate workers due to the worker's pregnancy, maternity leave, marriage, or breast-feeding of a child under 12 months of age or force them to resign? 
 Vietnam    721
Name: Country, dtype: int64
Sexual harassment 
 Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Is gender a factor in decisions regarding pay? 
 Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Nicaragua      119
Name: Country, dtype: int64
Is gender a factor in decisions regarding opportunities for promotion or access to training? 
 Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Nicaragua      119
Name: Country, 

In [155]:
df[df.Q=="Does the employer terminate workers who are pregnant or force them to resign?"].AssesmentDate.str[-4:].unique()

array(['2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype=object)

In [156]:
df[df.Q=="Does the employer terminate workers who are on maternity leave or force them to resign?"].AssesmentDate.str[-4:].unique()

array(['2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype=object)

In [81]:
df.loc[df.Q == "Does the employer dismiss workers or force them to resign if they are pregnant, on maternity leave or nursing? [differentiation question]", "QCleaned"] = "del"
df[df.Q=="Does the employer dismiss workers or force them to resign if they are pregnant, on maternity leave or nursing? [differentiation question]"].QCleaned.unique()

array(['del'], dtype=object)

In [86]:
for i in df[df.CPCleaned==CPCleanedUnique[2]].QCleaned.unique():
    print(i,"\n",getCountry(i))

Gender discrimination on conditions of work 
 ['Vietnam' 'Indonesia' 'Jordan' 'Nicaragua' 'Haiti' 'Cambodia'
 'Bangladesh']
Terminate workers due to pregnancy or maternity leave 
 ['Vietnam' 'Indonesia' 'Jordan' 'Nicaragua' 'Haiti' 'Cambodia'
 'Bangladesh']
Sexual harassment 
 ['Vietnam' 'Indonesia' 'Jordan' 'Nicaragua' 'Haiti' 'Cambodia'
 'Bangladesh']
Is the gender or marital status of a worker a factor in decisions regarding pay? 
 ['Vietnam']
Is the gender or marital status of a worker a factor in decisions regarding opportunities for promotion or access to training? 
 ['Vietnam']
Is the gender or marital status of the worker a factor in employer decisions regarding termination or retirement of workers? 
 ['Vietnam']
Does the employer require pregnancy tests or use of contraceptives as a condition of employment? 
 ['Vietnam']
Does the employer change the employment status, position, wages, benefits or seniority of workers during maternity leave? 
 ['Vietnam' 'Indonesia' 'Nicaragua'

In [334]:
CPCleanedUnique[3]

'Welfare Facilities'

In [157]:
#3:
df[df.CPCleaned==CPCleanedUnique[3]].QCleaned.value_counts()

Providing drinking water                                                                           6476
Does the workplace have adequate accessible toilets?                                               6252
Adequate washing facilities                                                                        4674
Does the workplace provide all workers a place to store their clothes?                             2688
Does the workplace have a functioning and accessible nursing room?                                 2383
Does employer provide cups or other sanitary means\nfor drinking water?                            2368
Does the workplace have adequate day care facilities?                                              2288
Adequate eating area                                                                               2118
Adequate hand washing taps                                                                         1802
Does the employer comply with requirements regarding canteen?   

In [335]:
tabCountry("Does employer provide cups or other sanitary means\nfor drinking water?")

Cambodia    2368
Name: Country, dtype: int64

In [333]:
2118+1802+53+474

4447

In [338]:
#4: cleaned
#5:
df[df.CPCleaned==CPCleanedUnique[5]].QCleaned.value_counts().sort_index()

Are chemical mixing rooms properly equipped and used?                                      2238
Chemicals/hazardous substance properly labelled                                            5164
Chemicals/hazardous substance properly stored                                              1802
Does the employer keep an inventory of hazardous chemicals used in the workplace?         12163
Has the employer appointed a Chemical Officer?                                              833
Has the employer prepared for response, control and cleaning of chemical spills?           8519
Has the employer trained all workers who work with chemicals and hazardous substances?     9412
Storage of chemicals and hazardous substances                                              6476
Name: QCleaned, dtype: int64

In [173]:
getOriginalQ("Has the employer trained all workers who work with chemicals and hazardous substances?").sort_index()

Has the employer effectively trained workers and supervisor who work with or are responsible for hazardous chemicals?                          249
Has the employer effectively trained workers and supervisors who work with or are responsible for hazardous chemicals?                        1553
Has the employer effectively trained workers who work with chemicals and hazardous substances?                                                3149
Has the employer provided workers who work with chemical substances with proper clothing and personal protective equipment when necessary?    1441
Has the employer taken action to  prevent and limit workers' exposure to chemicals and hazardous substances?                                  2290
Has the employer taken action to assess, monitor, prevent and limit workers' exposure to chemicals and hazardous substances?                   747
Has the employer taken action to prevent and limit workers' exposure to chemicals and hazardous substances?           

In [174]:
getOriginalQ("Does the employer keep an inventory of hazardous chemicals used in the workplace?").sort_index()

Does the employer have chemical safety data sheets for all the hazardous chemicals used in the workplace?                                                  161
Does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace, and are the chemicals properly labelled?             833
Does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?                                                     2982
Does the employer keep a chemical inventory, and ensure that all chemicals and hazardous substances are properly labelled and have safety data sheets?     474
Does the employer keep an inventory of chemicals and hazardous substances used in the workplace?                                                          3990
Does the employer keep an inventory of hazardous chemicals used in the workplace?                                                                         1802
Does the employer keep chemical safety records

In [177]:
#6: cleaned
#7: 
df[df.CPCleaned==CPCleanedUnique[7]].QCleaned.value_counts()

Does the employer pay workers correctly for personal leave?                                                                                                                    7524
Payment for maternity leave                                                                                                                                                    6474
Does the employer pay workers correctly for paid public holidays?                                                                                                              6069
Does the employer provide required breastfeeding breaks?                                                                                                                       4061
Paid annual leave                                                                                                                                                              3841
Does the employer pay workers correctly during suspensions of work?                                 

In [178]:
#8: 
df[df.CPCleaned==CPCleanedUnique[8]].QCleaned.value_counts()

Does the employer keep only one accurate payroll record?         6476
Has the employer made any unauthorized deductions from wages?    6476
Inform workers about wage pay and deduction                      5643
Name: QCleaned, dtype: int64

In [179]:
tabCountry("Inform workers about wage pay and deduction")

Cambodia      2503
Vietnam       1802
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64

In [187]:
#9:
df[df.CPCleaned==CPCleanedUnique[9]].QCleaned.value_counts().sort_index()

Are workers free to meet without management present?                                                                                                           6210
Does the employer provide incentives to workers to keep them from joining a union or engaging in union activities?                                             4674
Does the employer threaten, intimidate, or harass workers who join a union or engage in union activities?                                                      6476
Has the employer terminated a union official without getting permission from the Ministry?                                                                     2503
Has the employer terminated a union official without the written agreement of the union board or the higher-level union?                                       1775
Has the employer tried to interfere with, manipulate, or control the union(s)?                                                                                 1775
If there is more

In [188]:
#10:
df[df.CPCleaned==CPCleanedUnique[10]].QCleaned.value_counts().sort_index()

Are disciplinary measures proportional to the seriousness of workers' misconduct?                     2236
Bullying harassment                                                                                   6476
Did the employer resolve grievances and disputes in compliance with legal requirements?               6357
Do disciplinary measures comply with legal requirements?                                              3693
Do the disciplinary measures comply with the company's by-laws approved by the Ministry of Labor?      119
Does dialogue at the workplace take place in compliance with legal requirement?                       1802
Does the employer comply with requirements regarding shop stewards?                                   2411
Does the factory have a functioning Participation Committee?                                           959
Does the factory have a functioning bipartite cooperation institution?                                 833
Has the employer complied with arbitr

In [189]:
#11:
df[df.CPCleaned==CPCleanedUnique[11]].QCleaned.value_counts().sort_index()

Are workers free to terminate their employment with reasonable notice, and to leave their jobs when their contracts expire?                                  4930
Does the employer delay or withhold wage payments in order to coerce workers to work?                                                                        6092
Does the employer deny workers access to their personal documents (such as birth certificates, passports, work permits and ID cards) when they need them?    3380
Does the employer force workers to work to discipline them or as punishment for participation in a strike?                                                   6090
Does the employer restrict workers' freedom to come and go from the dormitories and/or the industrial park or zone in which the factory is located?          6476
Does the employer use threats such as deportation, cancellation of visas or reporting to the authorities in order to force workers to stay at the job?       3973
Does the employer use violen

In [192]:
#12: cleaned
#13:
df[df.CPCleaned==CPCleanedUnique[13]].QCleaned.value_counts().sort_index()

Adequate OSH policy                                                                                                                               2412
Does the employer conduct risk assessment?                                                                                                        1802
Does the employer develop an OSH plan annually?                                                                                                   2278
Does the employer have an OSH license?                                                                                                             119
Does the employer have legally required construction/building permits?                                                                            5150
Does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?     833
Does the employer inspect and measure the environmental conditions in the workplace on a annua

In [194]:
#14:
df[df.CPCleaned==CPCleanedUnique[14]].QCleaned.value_counts()

Minimum wages for regular workers                                                                                                                                                 6476
Paid piece rate workers correctluy                                                                                                                                                5160
Does the employer pay at least minimum wage for temporary and probationary workers?                                                                                               3678
Does the employer pay apprentices the wage agreed to by the parties?                                                                                                              2386
Is the piece rate set at a level that permits workers of average ability working normal hours to earn minimum wage?                                                               2284
Does the employer pay at least 7% higher than the applicable legal minimum wage for o

In [202]:
#15:
df[df.CPCleaned==CPCleanedUnique[15]].QCleaned.value_counts()

Punish workers for striking                                                                                                      6476
Has the employer hired new workers to replace striking workers during a strike?                                                  6209
prevent workers from strike?                                                                                                     6209
Has the employer called security guards, the police or armed forces to break up a peaceful strike or arrest striking workers?    1802
Name: QCleaned, dtype: int64

In [201]:
undo("Has the employer tried to prevent workers from participating in a strike?")
replace(["Has the employer tried to prevent any workers from participating in a strike?",\
         "Has the employer tried to prevent workers from participating in a strike?"],"prevent workers from strike?")

In [198]:
getCountry("Has the employer called security guards, the police or armed forces to break up a peaceful strike or arrest striking workers?")

array(['Vietnam'], dtype=object)

In [204]:
#16:
df[df.CPCleaned==CPCleanedUnique[16]].QCleaned.value_counts()

Implement CBA                                                                                                                                                       6476
Are the provisions of the collective agreement at least as favourable for workers as the law?                                                                       5839
Does the employer consult with unions where legally required?                                                                                                       5275
Does the employer provide workers access to the collective bargaining agreements?                                                                                   4038
Does the employer refuse to bargain collectively or refuse to bargain in good faith with the union, worker representatives, union federations or confederations?    3849
Is the grassroots level union in the factory involved in the bargaining process at the enterprise level?                                                   

In [205]:
#17:
df[df.CPCleaned==CPCleanedUnique[17]].QCleaned.value_counts()

Do regular daily working hours exceed 10 hours?                                                    6469
Do the working time records reflect the hours actually worked?                                     5368
Does the employer provide required weekly rest periods?                                            2836
Does the employer comply with daily break periods?                                                 2666
Do workers have at least one day off per week?                                                     2547
Does the employer comply with shift work requirements?                                             1804
Do the attendance records reflect the hours actually worked?                                        833
Does the employer post the factory's working hours (for all shifts, and including break times)?      81
Does the employer provide required breastfeeding breaks?                                             60
Name: QCleaned, dtype: int64

In [209]:
#18:
df[df.CPCleaned==CPCleanedUnique[18]].QCleaned.value_counts()

Does the employer pay all workers who work regularly the correct attendance bonus when workers take sick leave?                                                7312
Does the employer pay workers the correct seniority bonus?                                                                                                     2977
Does the employer pay workers the correct compensation for work-related accidents and diseases?                                                                2860
Does the employer pay all workers the correct mandatory wage supplements (including  transportation and housing allowances)?                                   2503
Does the employer pay the required employer contribution to the National Social Security Fund?                                                                 2503
Does the employer provide transport home or a place to sleep for workers who finish work between 22:00 and 05:00?                                              2250
Does the employe

In [210]:
#19:
df[df.CPCleaned==CPCleanedUnique[19]].QCleaned.value_counts()

Do the employment contracts comply with other legal requirements?                                            5846
Do workers understand the terms and conditions of employment?                                                5468
Do all persons who perform work for the factory, both on the premises and offsite, have a contract?          4940
Do the internal work rules comply with national law?                                                         4347
Do the contracts comply with the labour law, collective agreement and work rules?                            3629
Do the employment contracts comply with Cambodian labour law and with the factory's internal regulations?    2286
Has the employer provided all workers with a letter of appointment?                                          1752
Does the employer give a copy of the work agreement in Bahasa to workers?                                     833
Do the factory bylaws comply with Jordanian legal requirements and were they communicate

In [226]:
#20:
df[df.CPCleaned==CPCleanedUnique[20]].QCleaned.value_counts()

Does the employer comply with limits on the period of employment for training workers?                                                                                                     4627
Does the employer comply with the legal limit on the use of fixed term contracts?                                                                                                          2503
Does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?    2503
Does the employer comply with requirements for apprentices?                                                                                                                                2282
Do workers have to pay anyone to get a job?                                                                                                                                                2236
Does the employer comply with requiremen

In [224]:
tabCountry("Does the employer comply with limits on the period of employment for training workers?")

Cambodia      2241
Vietnam       1802
Bangladesh     584
Name: Country, dtype: int64

In [223]:
#21:
df[df.CPCleaned==CPCleanedUnique[21]].QCleaned.value_counts()

Does the employer only terminate workers for valid reasons?                                                                                           8158
Does the employer compensate workers for unused paid annual leave when they resign or are terminated?                                                 6357
Does the employer pay workers the correct severance pay?                                                                                              6313
Are workers given the opportunities required under law to defend themselves before they are terminated?                                               6196
Does the employer pay all outstanding wages and indemnities within 48 hours of termination?                                                           4674
Does the employer comply with legal requirements before suspending workers or reducing the size of the workforce due to changes in operations?        4578
Does the employer provide workers proper notice of termination?       

In [227]:
getOriginalQ("Does the employer only terminate workers for valid reasons?")

Does the employer only terminate workers for valid reasons?                                  4042
Does the employer unilaterally terminate workers for reasons that are not allowed by law?    1802
Does the employer dismiss workers for reasons that are not allowed by law?                   1801
Does the employer only terminate or dismiss workers for valid reasons?                        357
Does the employer terminate workers for reasons that are invalid under national law?          156
Name: Q, dtype: int64

In [232]:
df[df.Q=="Does the employer only terminate workers for valid reasons?"].Country.unique()

array(['Indonesia', 'Jordan', 'Haiti', 'Cambodia', 'Bangladesh'],
      dtype=object)

In [230]:
tabCountry("Does the employer only terminate or dismiss workers for valid reasons?")

Series([], Name: Country, dtype: int64)

In [234]:
#22:
df[df.CPCleaned==CPCleanedUnique[22]].QCleaned.value_counts()

Does the employer provide at least 12 weeks of maternity leave?                                                                 5801
Does the employer provide required breastfeeding breaks?                                                                        5623
Does the employer provide 12 days of annual leave per year?                                                                     4555
Does the employer provide 14 days of sick leave per year?                                                                       4473
Does the employer provide required annual leave?                                                                                4188
Does the employer pay for paternity leave when required?                                                                        2752
Do workers have at least one day off per week?                                                                                  2503
Are pregnant workers provided time off for prenatal visits?          

In [243]:
#23:
df[df.CPCleaned==CPCleanedUnique[23]].QCleaned.value_counts()

Do union representatives have access to workers in the workplace?                                                      6476
Does the employer deduct union dues from wages when workers request this in writing?                                   6175
Does the employer allow workers to carry out trade union activities during working hours as agreed by both parties?    2635
Does the employer provide a place to post the names, positions, and activities of the union leaders?                   2503
Does the employer provide workers access to the collective bargaining agreements?                                       584
Does the employer provide unions with access to bulletin boards in the workplace?                                       474
Name: QCleaned, dtype: int64

In [244]:
CPCleanedUnique[23]

'Union Operations'

In [245]:
df[df.QCleaned=="Does the employer provide workers access to the collective bargaining agreements?"].CPCleaned.unique()

array(['Collective Bargaining', 'Union Operations'], dtype=object)

In [240]:
for i in df[df.CPCleaned==CPCleanedUnique[23]].QCleaned.unique():
    print(i,"\n",tabCountry(i))


Do union representatives have access to workers in the workplace? 
 Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Does the employer deduct union dues from wages when workers request this in writing? 
 Cambodia      2240
Vietnam       1775
Indonesia      833
Bangladesh     584
Jordan         463
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Does the employer allow workers to carry out trade union activities during working hours as agreed by both parties? 
 Vietnam      1802
Indonesia     833
Name: Country, dtype: int64
Does the employer provide unions with access to bulletin boards in the workplace? 
 Jordan    474
Name: Country, dtype: int64
Does the employer provide a place to post the names, positions, and activities of the union leaders? 
 Cambodia    2503
Name: Country, dtype: int64
Does the employer provide workers access to the collective bargaini

In [246]:
#24:
df[df.CPCleaned==CPCleanedUnique[24]].QCleaned.value_counts()

Do workers use the machines and equipment in a safe manner?                                                                                             8892
Safety warnings posted                                                                                                                                  6476
Electrical wires/switches/plugs properly installed                                                                                                      6476
Proper guards installed and maintained                                                                                                                  6476
Does the employer comply with ergonomic requirements?                                                                                                   6287
Provide PPE                                                                                                                                             5035
Protect workers from falls from heights                   

In [252]:
#25:
df[df.CPCleaned==CPCleanedUnique[25]].QCleaned.value_counts()

Is the workplace adequately lit?                                                             6416
Are noise levels acceptable?                                                                 6344
Is the workplace clean and tidy?                                                             5411
Is the temperature and ventilation aceptable?                                                3107
Is the temperature in the workplace acceptable?                                              2432
Is the workplace adequately ventilated?                                                      1809
Is the workplace climate measured by the employer, and in line with applicable standards?     833
Name: QCleaned, dtype: int64

In [251]:
replace(["Are noise levels acceptable?","Are noise levels  measured by the employer and in line with applicable standard?"])

In [253]:
for i in df[df.CPCleaned==CPCleanedUnique[25]].QCleaned.unique():
    print(i,"\n",tabCountry(i))

Is the temperature in the workplace acceptable? 
 Vietnam    1802
Jordan      474
Haiti       156
Name: Country, dtype: int64
Is the workplace clean and tidy? 
 Cambodia      2271
Vietnam       1802
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Is the workplace adequately ventilated? 
 Vietnam       3604
Indonesia      833
Bangladesh     584
Haiti          166
Cambodia         3
Nicaragua        2
Name: Country, dtype: int64
Are noise levels acceptable? 
 Cambodia      2371
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Is the workplace adequately lit? 
 Cambodia      2443
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Is the workplace climate measured by the employer, and in line with applicable standards? 
 Indonesia    833
Name: Country, d

In [256]:
set([1,None,None])

{1, None}

In [ ]:
def find(qkw=None,country=None,year=None)
    if 
    df[(df.QCleaned.str.contains(qkw)) & ()]

In [257]:
#26:
df[df.CPCleaned==CPCleanedUnique[26]].QCleaned.value_counts()

Does the employer provide adequate first-aid facilities?                             7077
Comply with HIV/AIDS policy                                                          5841
Does the employer address safety and health risks to pregnant or nursing workers?    5735
Trained workers on first aid                                                         5426
Does the workplace have an adequately equipped and staffed infirmary?                2503
Does the workplace have all required facilities?                                     2437
Do workers undergo a medical examination before being employed?                      2386
Do workers have annual medical checks?                                               2240
Do workers receive free periodic health checks as required by law?                   1963
Does the employer provide required medical services?                                 1417
Trained workers on first aid                                                          833
Do workers

In [258]:
#27: cleaned*
df[df.CPCleaned==CPCleanedUnique[27]].QCleaned.value_counts()

Do workers who are under age 18 work overtime or work at night?              5929
Do workers who are under age 18 perform work that is hazardous by nature?    5616
Name: QCleaned, dtype: int64

In [259]:
for i in df[df.CPCleaned==CPCleanedUnique[27]].QCleaned.unique():
    print(i,"\n",tabCountry(i))

Do workers who are under age 18 work overtime or work at night? 
 Cambodia      2236
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Name: Country, dtype: int64
Do workers who are under age 18 perform work that is hazardous by nature? 
 Cambodia      2236
Vietnam       1802
Indonesia      833
Bangladesh     584
Haiti          161
Name: Country, dtype: int64


In [263]:
#28: cleaned*
df[df.CPCleaned==CPCleanedUnique[28]].QCleaned.value_counts()

Have you found that religion or political opinion is a factor in the employer's decisions regarding termination or retirement of workers?    1802
Are workers harassed on the basis of their religion or political opinion?                                                                    1802
Name: QCleaned, dtype: int64

In [265]:
#29:
df[df.CPCleaned==CPCleanedUnique[29]].QCleaned.value_counts()

Are workers paid on time, twice per month?                                            2878
Are workers' full wages paid directly to workers at the workplace on working days?    2404
Does the employer pay wages on time and pay interest in case of late payment?         1802
Are wages paid on time?                                                               1113
Are workers paid within seven days from the date wages become payable?                 474
Does the employer comply with other wage payments?                                     474
Name: QCleaned, dtype: int64

In [266]:
#30:
df[df.CPCleaned==CPCleanedUnique[30]].QCleaned.value_counts()

Is the accommodation separate from the workplace (even though it may be in the same compound/industrial park)?                 6091
Has the employer adequately prepared for emergencies in the accommodation?                                                     5651
Is the accommodation protected against fire?                                                                                   5619
Adequate toilets, shower, sewage in dorm                                                                                       3857
Is the accommodation protected against disease carrying animals or insects?                                                    3857
Does the accommodation offer workers adequate privacy?                                                                         3857
Is the workplace adequately ventilated?                                                                                        3383
Does the accommodation have adequate cooking facilities?                    

In [267]:
#31:
df[df.CPCleaned==CPCleanedUnique[31]].QCleaned.value_counts()

Is an applicant's race, colour or origin a factor in hiring decisions?                                          11171
Is race, colour or origin a factor in decisions regarding opportunities for promotion or access to training?     5643
Is race, colour, origin or citizenship status a factor in decisions regarding pay?                                474
Name: QCleaned, dtype: int64

In [280]:
collapse(CPCleanedUnique[31])

Is an applicant's race, colour or origin a factor in hiring decisions?
Cambodia      5006
Vietnam       3604
Bangladesh    1168
Indonesia      833
Haiti          322
Nicaragua      238
Name: Country, dtype: int64
---------------------------
11171
Is race, colour or origin a factor in decisions regarding opportunities for promotion or access to training?
Cambodia      2503
Vietnam       1802
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
---------------------------
5643
Is race, colour, origin or citizenship status a factor in decisions regarding pay?
Jordan    474
Name: Country, dtype: int64
---------------------------
474


In [271]:
#32:
df[df.CPCleaned==CPCleanedUnique[32]].QCleaned.value_counts()

Has the employer made required accommodations for physically disabled workers?                                                                             3109
Does the employer hire one disabled worker for every 100 workers?                                                                                           833
Is an applicant's age a factor in hiring decisions?                                                                                                         721
Do recruitment materials such as job announcements or job application forms refer to age in a manner that could discourage older workers from applying?     721
Is age a factor in decisions regarding opportunities for promotion or access to training?                                                                   721
Name: QCleaned, dtype: int64

In [272]:
#33:
df[df.CPCleaned==CPCleanedUnique[33]].QCleaned.value_counts()

Is overtime work limited to two hours per day?                                                     9546
Is overtime voluntary?                                                                             6476
Does the employer get permission from the Labour Inspector before workers work overtime?           2458
Does the employer get permission from the Labor Inspector before suspending weekly rest days?      2276
Does the employer provide required weekly rest periods?                                            1802
Does the employer prepare written instructions on overtime?                                        1417
Do workers have at least one day off per week?                                                      584
Does the employer get written consent from women to work at night?                                  584
Does the employer obtain authorization from the Department of Labour before working on Sundays?      80
Name: QCleaned, dtype: int64

In [275]:
collapse(CPCleanedUnique[33])

Is overtime voluntary?
Cambodia      2503
Vietnam       1802
Indonesia      833
Bangladesh     584
Jordan         474
Haiti          161
Nicaragua      119
Name: Country, dtype: int64
Does the employer provide required weekly rest periods?
Cambodia     2243
Vietnam      1802
Jordan        474
Nicaragua     119
Name: Country, dtype: int64
Is overtime work limited to two hours per day?
Vietnam       5406
Cambodia      2503
Indonesia      833
Bangladesh     584
Nicaragua      119
Haiti          101
Name: Country, dtype: int64
Does the employer prepare written instructions on overtime?
Indonesia     833
Bangladesh    584
Name: Country, dtype: int64
Does the employer get permission from the Labour Inspector before workers work overtime?
Cambodia     2259
Nicaragua     119
Haiti          80
Name: Country, dtype: int64
Does the employer obtain authorization from the Department of Labour before working on Sundays?
Haiti    80
Name: Country, dtype: int64
Does the employer get permission from th

In [273]:
#34:
df[df.CPCleaned==CPCleanedUnique[34]].QCleaned.value_counts()

Does the employer reliably verify the age of workers prior to hiring?                                                                 5050
Does the employer keep a record of workers under 18 years of age?                                                                     4784
Does the employer comply with documentation requirements for workers under age of 18?                                                 1307
Does the employer have a verification system in place to ensure they do not hire child labour in accordance with the National Law?     833
Does the employer comply with requirements regarding the certificate of fitness for workers under age 18?                              584
Name: QCleaned, dtype: int64

----